# ISIBrno-AIMT
ISIBrno-AIMT is the winner of the [Will Two Do?](https://physionet.org/content/challenge-2021/1.0.3/sources/) challenge from [physionet.org](https://physionet.org/) 

[Source Code](https://physionet.org/static/published-projects/challenge-2021/1.0.3/sources/ISIBrnoAIMT.zip)
[Paper](https://www.cinc.org/archives/2021/pdf/CinC2021-014.pdf)
[leaderboard](https://moody-challenge.physionet.org/2021/results/)

In [1]:
import pickle
import os
import sys
import torch
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, jaccard_score, confusion_matrix, precision_score, recall_score, accuracy_score

from models.m02_ISIBrnoAIMT_BagOfWords.train import *
from models.m02_ISIBrnoAIMT_BagOfWords.model import NN
from models.m02_ISIBrnoAIMT_BagOfWords.dataset import PtbXlDataset

In [2]:
os.chdir('..')

### Bag of Words with 50 Words

In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_BoW = 50
dataset = PtbXlDataset('data_ptb-xl/', 'test', n_BoW)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

model = NN(n_BoW).to(device)
model.load_state_dict(torch.load(f'models/m02_ISIBrnoAIMT_BagOfWords/model_{n_BoW}_BoW.pt'))

<All keys matched successfully>

In [9]:
y, pred = get_predictions(model, dataloader)
bow_columns = pd.read_csv('data_ptb-xl/bag_of_words/train_50_BoW.csv').columns[1:]

In [10]:
y = pd.DataFrame(y, columns=bow_columns)
pred = pd.DataFrame(pred, columns=bow_columns)

In [11]:
f1 = f1_score(y, pred, average='samples')
iou = jaccard_score(y, pred, average='samples')

print(f'F1: {f1}\nIOU: {iou}')

F1: 0.23093855881219758
IOU: 0.1354810696408539


In [12]:
performance_data = []

# Iterate over each label in the DataFrame
for label in y.columns:
    # Compute confusion matrix for the current label
    tn, fp, fn, tp = confusion_matrix(y[label], pred[label]).ravel()
    f1 = f1_score(y[label], pred[label])
    precision = precision_score(y[label], pred[label], zero_division=0)
    recall = recall_score(y[label], pred[label])
    accuracy = accuracy_score(y[label], pred[label])
    
    # Append the results to the list
    performance_data.append({'Label': label, 'f1': f1, 'precision': precision, 'recall': recall, 'accuracy': accuracy, 'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn})

# Convert the list of dicts into a DataFrame
performance_summary = pd.DataFrame(performance_data)

performance_summary

,Label,f1,precision,recall,accuracy,TP,TN,FP,FN
0,46,0.432674,0.403571,0.466300,0.590794,678,1889,1002,776
1,abnormal,0.325033,0.340659,0.310777,0.644419,372,2428,720,825
2,anterior,0.095783,0.066866,0.168766,0.708861,67,3013,935,330
3,anterolateral,0.079127,0.042336,0.604167,0.223245,145,825,3280,95
4,anteroseptal,0.072289,0.045215,0.180180,0.645570,60,2745,1267,273
5,atrial,0.112676,0.085393,0.165577,0.724511,76,3072,814,383
6,avl,0.080864,0.043375,0.595918,0.236133,146,880,3220,99
7,axis,0.202851,0.116646,0.777311,0.665362,185,2706,1401,53
8,block,0.306185,0.181335,0.982962,0.217722,750,196,3386,13
9,branch,0.202683,0.113306,0.959677,0.138090,476,124,3725,20


### Bag of Words with 20 Words

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_BoW = 20
dataset = PtbXlDataset('data_ptb-xl/', 'test', n_BoW)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

model = NN(n_BoW).to(device)
model.load_state_dict(torch.load(f'models/m02_ISIBrnoAIMT_BagOfWords/model_{n_BoW}_BoW.pt'))

<All keys matched successfully>

In [4]:
y, pred = get_predictions(model, dataloader)
bow_columns = pd.read_csv('data_ptb-xl/bag_of_words/train_20_BoW.csv').columns[1:]

In [5]:
y = pd.DataFrame(y, columns=bow_columns)
pred = pd.DataFrame(pred, columns=bow_columns)

In [6]:
f1 = f1_score(y, pred, average='samples')
iou = jaccard_score(y, pred, average='samples')

print(f'F1: {f1}\nIOU: {iou}')

F1: 0.35301590569169544
IOU: 0.22702074991139856


In [7]:
performance_data = []

# Iterate over each label in the DataFrame
for label in y.columns:
    # Compute confusion matrix for the current label
    tn, fp, fn, tp = confusion_matrix(y[label], pred[label]).ravel()
    f1 = f1_score(y[label], pred[label])
    precision = precision_score(y[label], pred[label], zero_division=0)
    recall = recall_score(y[label], pred[label])
    accuracy = accuracy_score(y[label], pred[label])
    
    # Append the results to the list
    performance_data.append({'Label': label, 'f1': f1, 'precision': precision, 'recall': recall, 'accuracy': accuracy, 'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn})

# Convert the list of dicts into a DataFrame
performance_summary = pd.DataFrame(performance_data)

performance_summary

,Label,f1,precision,recall,accuracy,TP,TN,FP,FN
0,46,0.478036,0.326690,0.890646,0.349137,1295,222,2669,159
1,abnormal,0.290736,0.225926,0.407686,0.452014,488,1476,1672,709
2,block,0.343750,0.242343,0.591088,0.603682,451,2172,1410,312
3,ecg,0.312191,0.254420,0.403905,0.328654,662,766,1940,977
4,infarction,0.050465,0.035514,0.087156,0.506329,57,2143,1548,597
5,inferior,0.242637,0.139591,0.926786,0.254315,519,586,3199,41
6,lead,0.237918,0.160200,0.462094,0.622555,256,2449,1342,298
7,left,0.451314,0.318430,0.774536,0.346375,1168,337,2500,340
8,non,0.200084,0.111371,0.983505,0.122209,477,54,3806,8
9,normal,0.680831,0.832458,0.575929,0.692290,1426,1582,287,1050
